### TensorFlow
- 분산처리 기능 강화
    - 다른 컴퓨터와 통신을 해야함. (Session 기능추가)

##### 1. 심볼변수 정의

In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [3]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

##### 2. 심볼관계 정의

In [4]:
z = x + y

In [5]:
type(z)

tensorflow.python.framework.ops.Tensor

In [6]:
u = tf.log(z)
type(u)

tensorflow.python.framework.ops.Tensor

##### 2. Session
- function이 아닌 session으로 한다.

In [7]:
sess = tf.Session()
init = tf.global_variables_initializer() # 각 심볼에 들어갈 초기값이 셋팅
sess.run(init)
print(sess.run(z))
print(sess.run(u))
sess.close()

3.0
1.0986123


##### 3. 미분

In [8]:
f = x**2
fx = tf.gradients(f, [x])
# with구문으로 close가능
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))    
    print(sess.run(fx))

1.0
[2.0]


##### 4. 최적화
- GradientDescentOptimizer 등의 클래스를 통해 update그래프가 튀어나온다.

##### 5. Tensorboard라는 웹 서버
- vector, matrix summary는 없음. 대신 히스토그램 제공(scalar summary는 있음, 이미지, 음성도 따로 summary존재)
- summary.merge : 원하는 summary들을 하나의 summary로 합침

In [9]:
!rm -rf /home/dockeruser/logs/*

In [24]:
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] * 0.5)[:, np.newaxis] # y=1 또는 y=0. 2차원임을 명시

In [131]:
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2,1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
d = tf.nn.sigmoid(tf.matmul(X_data, w) + b)

# zero = np.zeros(1, dtype=np.float64)[0] # float64 자료형의 0 값 생성 
cost = tf.reduce_sum((y_data - d)**2)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cost)

sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.histogram("b", b)
d_summary = tf.summary.histogram("d", d)
cost_summary = tf.summary.histogram("cost", cost)
merged = tf.summary.merge([w_summary, b_summary, d_summary, cost_summary])


init = tf.global_variables_initializer()
sess.run(init)

writer = tf.summary.FileWriter("C:/home/dockeruser/log", sess.graph)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  26.290276373227286
  10:  15.5158459529319
  20:  43.06665688541476
  30:  42.91550316994202
  40:  38.494576725459034
  50:  9.009603111032
  60:  16.26356489306519
  70:  8.208568530968687
  80:  2.0904292198174517
  90:  2.0311847954025946


In [132]:
y_pred = sess.run(tf.round(d))

In [133]:
sess.close()

In [134]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 0, 50]], dtype=int64)